In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim

C:\Users\haho6\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
X = tf.placeholder(tf.float32, [None, 64,64,3])
Y = tf.placeholder(tf.float32, [None, 10])
Z = tf.placeholder(tf.float32, [None, 100])

In [3]:
def GENERATOR(Z):
    x = slim.fully_connected(Z, (8 * 8 * 16), activation_fn=None)
    x = tf.reshape(x, [-1, 8, 8, 16])
    for i in range(3):
        x = slim.conv2d(x, 16, 3, 1, activation_fn=tf.nn.elu)
        x = slim.conv2d(x, 16, 3, 1, activation_fn=tf.nn.elu)
        x = upscale(x, 2)
    x = slim.conv2d(x, 3, 3, 1, activation_fn=None)
    return x

In [4]:
def upscale(x, scale):
    _, h, w, _ = int_shape(x)
    return tf.image.resize_nearest_neighbor(x, (h*scale, w*scale))

def int_shape(tensor):
    shape = tensor.get_shape().as_list()
    return [num if num is not None else -1 for num in shape]

def norm_img(image):
    return image/127.5 - 1.

def denorm_img(norm):    
    return tf.clip_by_value(((norm + 1)*127.), 0, 255)

def sample_z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [5]:
def DISCRIMINATOR(X):
    with tf.variable_scope('D', reuse=tf.AUTO_REUSE) as vs:
        x = slim.conv2d(X, 16, 3, 1, activation_fn=tf.nn.elu)
        x = slim.conv2d(x, 16, 3, 1, activation_fn=tf.nn.elu)
        x = slim.conv2d(x, 16, 3, 1, activation_fn=tf.nn.elu) # 64 * 16
        x = slim.conv2d(x, 32, 3, 2, activation_fn=tf.nn.elu) 

        x = slim.conv2d(x, 32, 3, 1, activation_fn=tf.nn.elu)
        x = slim.conv2d(x, 32, 3, 1, activation_fn=tf.nn.elu) # 32 * 32
        x = slim.conv2d(x, 48, 3, 2, activation_fn=tf.nn.elu) 

        x = slim.conv2d(x, 48, 3, 1, activation_fn=tf.nn.elu)
        x = slim.conv2d(x, 48, 3, 1, activation_fn=tf.nn.elu) # 16 * 48

        x = tf.layers.flatten(x)
        z = slim.fully_connected(x, 100, activation_fn=None)
    return z

In [6]:
G = GENERATOR(Z)

RD = DISCRIMINATOR(X)
GD = DISCRIMINATOR(G)

_RD = GENERATOR(RD)
_GD = GENERATOR(GD)

L_REAL = tf.reduce_mean(tf.abs(_RD - X))
L_GENE = tf.reduce_mean(tf.abs(_GD - X))

K = tf.placeholder(tf.float32)

In [7]:
# BEGAN

D_LOSS = L_REAL - K*L_GENE
G_LOSS = tf.reduce_mean(tf.abs(_GD - X))


D_solver = (tf.train.AdamOptimizer(learning_rate=0.001)
            .minimize(D_LOSS))
G_solver = (tf.train.AdamOptimizer(learning_rate=0.001)
            .minimize(G_LOSS))


In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [33]:
# BEGAN
K_NOW = 0.001
gamma = 0.3
lam = 1e-3

for i in range(2000):
    NOISE = sample_z(16,100)
    BAT_IMAGE = images[0:16]
    _, L_REAL_NOW = sess.run([D_solver, L_REAL], 
                  feed_dict={
                      X: BAT_IMAGE,
                      Z: NOISE,
                      K: K_NOW })
    _, L_GENE_NOW = sess.run([G_solver, L_GENE], 
                  feed_dict={
                      X: BAT_IMAGE,
                      Z: NOISE})
    
    K_NOW = K_NOW + lam * (gamma*L_REAL_NOW - L_GENE_NOW)
    measure = L_GENE_NOW + np.abs(gamma*L_REAL_NOW - L_GENE_NOW)
    print('Iter-{}; Convergence measure: {:.4} D Loss: {:.4}'.format(i, measure, L_REAL_NOW))

    if i % 10 == 0:
        made_image = sess.run(G, feed_dict={Z:sample_z(16,100)})
        np.random.shuffle(images)
        c = 0
        for j in made_image[1:3]:
            c += 1
            cv2.imwrite('%03d-%02d.jpg'%(i,c),(j+1)*120)

Iter-0; Convergence measure: 0.8053 D Loss: 0.2109
Iter-1; Convergence measure: 0.7689 D Loss: 0.2185
Iter-2; Convergence measure: 0.7171 D Loss: 0.2176
Iter-3; Convergence measure: 0.6985 D Loss: 0.2127
Iter-4; Convergence measure: 0.7034 D Loss: 0.2126
Iter-5; Convergence measure: 0.7091 D Loss: 0.2109
Iter-6; Convergence measure: 0.725 D Loss: 0.2088
Iter-7; Convergence measure: 0.7503 D Loss: 0.2106
Iter-8; Convergence measure: 0.7364 D Loss: 0.2101
Iter-9; Convergence measure: 0.7056 D Loss: 0.2076
Iter-10; Convergence measure: 0.6941 D Loss: 0.2077
Iter-11; Convergence measure: 0.6765 D Loss: 0.215
Iter-12; Convergence measure: 0.6774 D Loss: 0.212
Iter-13; Convergence measure: 0.6665 D Loss: 0.21
Iter-14; Convergence measure: 0.6524 D Loss: 0.2091
Iter-15; Convergence measure: 0.6397 D Loss: 0.2069
Iter-16; Convergence measure: 0.6332 D Loss: 0.205
Iter-17; Convergence measure: 0.6371 D Loss: 0.2042
Iter-18; Convergence measure: 0.6415 D Loss: 0.2032
Iter-19; Convergence measure

Iter-158; Convergence measure: 0.6342 D Loss: 0.1941
Iter-159; Convergence measure: 0.6321 D Loss: 0.1936
Iter-160; Convergence measure: 0.6402 D Loss: 0.1931
Iter-161; Convergence measure: 0.6357 D Loss: 0.2169
Iter-162; Convergence measure: 0.6382 D Loss: 0.214
Iter-163; Convergence measure: 0.6348 D Loss: 0.2095
Iter-164; Convergence measure: 0.6345 D Loss: 0.2051
Iter-165; Convergence measure: 0.6327 D Loss: 0.2027
Iter-166; Convergence measure: 0.6315 D Loss: 0.2022
Iter-167; Convergence measure: 0.6352 D Loss: 0.2021
Iter-168; Convergence measure: 0.632 D Loss: 0.2019
Iter-169; Convergence measure: 0.629 D Loss: 0.201
Iter-170; Convergence measure: 0.6329 D Loss: 0.1996
Iter-171; Convergence measure: 0.6442 D Loss: 0.2071


KeyboardInterrupt: 

In [133]:

NOISE= _RD
made_image = sess.run(G, feed_dict={
                      X: BAT_IMAGE,
                      Z: NOISE,
                      K: K_NOW }
                     )
c = 0
i = 0
for j in BAT_IMAGE[:5]:
    c += 1
    cv2.imwrite('N%03d-%02d.jpg'%(i,c),cv2.cvtColor((j+1)*120, cv2.COLOR_RGB2BGR))

In [128]:
G = GENERATOR(Z)

RD = DISCRIMINATOR(X)
GD = DISCRIMINATOR(G)


L_REAL = tf.reduce_mean(tf.abs(G - X))
L_GENE = tf.reduce_mean(tf.abs(G - X))

K = tf.placeholder(tf.float32)

# BEGAN

D_LOSS = L_REAL - K*L_GENE
G_LOSS = tf.reduce_mean(tf.abs(G - X))


D_solver = (tf.train.AdamOptimizer(learning_rate=0.001)
            .minimize(D_LOSS))
G_solver = (tf.train.AdamOptimizer(learning_rate=0.001)
            .minimize(G_LOSS))

K_NOW = 0.001
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [132]:
gamma = 0.7
for i in range(1000):
    NOISE = sample_z(16,100)
    BAT_IMAGE = images[0:16]
    _, L_REAL_NOW, _RD = sess.run([D_solver, L_REAL, RD], 
                  feed_dict={
                      X: BAT_IMAGE,
                      Z: NOISE,
                      K: K_NOW })
    _, L_GENE_NOW = sess.run([G_solver, L_GENE], 
                  feed_dict={
                      X: BAT_IMAGE,
                      Z: _RD})
    
    K_NOW = K_NOW + lam * (gamma*L_REAL_NOW - L_GENE_NOW)
    measure = L_GENE_NOW + np.abs(gamma*L_REAL_NOW - L_GENE_NOW)
    print('Iter-{}; Convergence measure: {:.4} D Loss: {:.4}'.format(i, measure, L_REAL_NOW))

    if i % 10 == 0:
        made_image = sess.run(G, feed_dict={Z:sample_z(16,100)})
        np.random.shuffle(images)
        c = 0
        for j in made_image[1:3]:
            c += 1
            cv2.imwrite('%03d-%02d.jpg'%(i,c),cv2.cvtColor((j+1)*120, cv2.COLOR_RGB2BGR))

Iter-0; Convergence measure: 0.3428 D Loss: 0.4897
Iter-1; Convergence measure: 0.3189 D Loss: 0.4556
Iter-2; Convergence measure: 0.3532 D Loss: 0.4079
Iter-3; Convergence measure: 0.36 D Loss: 0.3785
Iter-4; Convergence measure: 0.346 D Loss: 0.3747
Iter-5; Convergence measure: 0.3448 D Loss: 0.3761
Iter-6; Convergence measure: 0.3554 D Loss: 0.3751
Iter-7; Convergence measure: 0.3703 D Loss: 0.3691
Iter-8; Convergence measure: 0.3774 D Loss: 0.3678
Iter-9; Convergence measure: 0.3775 D Loss: 0.3721
Iter-10; Convergence measure: 0.3787 D Loss: 0.374
Iter-11; Convergence measure: 0.3834 D Loss: 0.4334
Iter-12; Convergence measure: 0.397 D Loss: 0.4248
Iter-13; Convergence measure: 0.4062 D Loss: 0.4067
Iter-14; Convergence measure: 0.3876 D Loss: 0.3936
Iter-15; Convergence measure: 0.3538 D Loss: 0.4017
Iter-16; Convergence measure: 0.3484 D Loss: 0.4075
Iter-17; Convergence measure: 0.327 D Loss: 0.4143
Iter-18; Convergence measure: 0.3319 D Loss: 0.4014
Iter-19; Convergence measure

Iter-158; Convergence measure: 0.2747 D Loss: 0.3637
Iter-159; Convergence measure: 0.2719 D Loss: 0.3657
Iter-160; Convergence measure: 0.2709 D Loss: 0.365
Iter-161; Convergence measure: 0.2774 D Loss: 0.3755
Iter-162; Convergence measure: 0.2724 D Loss: 0.382
Iter-163; Convergence measure: 0.2736 D Loss: 0.3778
Iter-164; Convergence measure: 0.2725 D Loss: 0.3763
Iter-165; Convergence measure: 0.2672 D Loss: 0.3804
Iter-166; Convergence measure: 0.2666 D Loss: 0.3774
Iter-167; Convergence measure: 0.2666 D Loss: 0.3739
Iter-168; Convergence measure: 0.2636 D Loss: 0.3745
Iter-169; Convergence measure: 0.2629 D Loss: 0.3755
Iter-170; Convergence measure: 0.265 D Loss: 0.3785
Iter-171; Convergence measure: 0.2744 D Loss: 0.3727
Iter-172; Convergence measure: 0.2704 D Loss: 0.377
Iter-173; Convergence measure: 0.2717 D Loss: 0.374
Iter-174; Convergence measure: 0.2717 D Loss: 0.372
Iter-175; Convergence measure: 0.2708 D Loss: 0.3713
Iter-176; Convergence measure: 0.2706 D Loss: 0.3689

Iter-314; Convergence measure: 0.257 D Loss: 0.3672
Iter-315; Convergence measure: 0.2537 D Loss: 0.3624
Iter-316; Convergence measure: 0.2534 D Loss: 0.3621
Iter-317; Convergence measure: 0.2531 D Loss: 0.3616
Iter-318; Convergence measure: 0.2537 D Loss: 0.3624
Iter-319; Convergence measure: 0.2516 D Loss: 0.3594
Iter-320; Convergence measure: 0.254 D Loss: 0.3629
Iter-321; Convergence measure: 0.2499 D Loss: 0.357
Iter-322; Convergence measure: 0.2495 D Loss: 0.3564
Iter-323; Convergence measure: 0.2504 D Loss: 0.3577
Iter-324; Convergence measure: 0.2478 D Loss: 0.3541
Iter-325; Convergence measure: 0.2481 D Loss: 0.3544
Iter-326; Convergence measure: 0.2467 D Loss: 0.3524
Iter-327; Convergence measure: 0.247 D Loss: 0.3528
Iter-328; Convergence measure: 0.2467 D Loss: 0.3524
Iter-329; Convergence measure: 0.2467 D Loss: 0.3525
Iter-330; Convergence measure: 0.246 D Loss: 0.3515
Iter-331; Convergence measure: 0.2454 D Loss: 0.3506
Iter-332; Convergence measure: 0.2459 D Loss: 0.351

Iter-470; Convergence measure: 0.2592 D Loss: 0.3702
Iter-471; Convergence measure: 0.2285 D Loss: 0.3265
Iter-472; Convergence measure: 0.2289 D Loss: 0.327
Iter-473; Convergence measure: 0.2289 D Loss: 0.327
Iter-474; Convergence measure: 0.2304 D Loss: 0.3291
Iter-475; Convergence measure: 0.2297 D Loss: 0.3281
Iter-476; Convergence measure: 0.2279 D Loss: 0.3255
Iter-477; Convergence measure: 0.2271 D Loss: 0.3245
Iter-478; Convergence measure: 0.2265 D Loss: 0.3235
Iter-479; Convergence measure: 0.2266 D Loss: 0.3237
Iter-480; Convergence measure: 0.227 D Loss: 0.3243
Iter-481; Convergence measure: 0.2362 D Loss: 0.3374
Iter-482; Convergence measure: 0.2348 D Loss: 0.3355
Iter-483; Convergence measure: 0.235 D Loss: 0.3357
Iter-484; Convergence measure: 0.2349 D Loss: 0.3356
Iter-485; Convergence measure: 0.2353 D Loss: 0.3361
Iter-486; Convergence measure: 0.2356 D Loss: 0.3366
Iter-487; Convergence measure: 0.2343 D Loss: 0.3347
Iter-488; Convergence measure: 0.2333 D Loss: 0.33

KeyboardInterrupt: 

In [131]:
from keras.preprocessing.image import load_img, img_to_array
import keras
import tensorflow as tf
import numpy as np
import cv2
import os
import math
files = os.listdir(r'..\AnimeFaceDetecter\Datas\result\miku')
keras.backend.set_image_data_format('channels_last')
#keras.backend.set_image_data_format('channels_first')
images = []
for i in files:
    image = load_img(r'..\AnimeFaceDetecter\Datas\result\miku\{}'.format(i),False,target_size=(64,64))
    image = img_to_array(image)
    image = image
    images.append(norm_img(image))
print(image.shape)

(64, 64, 3)


<tf.Tensor 'Conv_6/Elu:0' shape=(?, 64, 64, 3) dtype=float32>

In [3]:
import numpy as np

In [6]:
images= np.zeros([16,64,64,3])